In [11]:
# 00 ... install base OS and application libraries
# https://aws.amazon.com/blogs/machine-learning/load-and-transform-data-from-delta-lake-using-amazon-sagemaker-studio-and-apache-spark/
%conda install openjdk -q -y

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [12]:
# 01 ... install base Python libraries
%pip install boto3
%pip install sagemaker
%pip install sagemaker_pyspark
%pip install pyarrow
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:
# 02 ... import Python libraries and configure
import boto3
import copy
import io
import json
import numpy
import os
import os.path
import pandas
import sagemaker
import sys
import time
import seaborn
import matplotlib.pyplot as plt

from sagemaker import get_execution_role

# session variables
role = get_execution_role()
session = sagemaker.Session()

# S3 bucket and prefix
bucket = session.default_bucket()

print(bucket)

sagemaker-us-east-2-645411899653


In [14]:
#02 ... download data about Lending Club Loans
# https://www.kaggle.com/datasets/wordsforthewise/lending-club?resource=download

def download_lending_club():    
    s3 = boto3.client("s3", region_name="us-east-2")
    # check if file does not yet exist
    # rejected_2007_to_2018Q4.csv.gz
    rejected_file_exists = os.path.exists("rejected_2007_to_2018Q4.csv.gz")
    if (not rejected_file_exists):
        s3.download_file("sagemaker-us-east-2-645411899653", "bigdata-demos/lendingclub/input/rejected_2007_to_2018Q4.csv.gz", "rejected_2007_to_2018Q4.csv.gz")
        
    accepted_file_exists = os.path.exists("accepted_2007_to_2018Q4.csv.gz")
    if (not accepted_file_exists):
        s3.download_file("sagemaker-us-east-2-645411899653", "bigdata-demos/lendingclub/input/accepted_2007_to_2018Q4.csv.gz", "accepted_2007_to_2018Q4.csv.gz")
    
    # gunzip
    !gunzip --force rejected_2007_to_2018Q4.csv.gz
    !gunzip --force accepted_2007_to_2018Q4.csv.gz

download_lending_club()

In [10]:
# 03 ... setup PySpark session
import sagemaker_pyspark
from pyspark.sql import SparkSession, DataFrame
from pyspark.ml import Pipeline

classpath = ":".join(sagemaker_pyspark.classpath_jars())
spark = (
    SparkSession.builder.config("spark.driver.extraClassPath", classpath)
    .master("local[*]")
    .getOrCreate()
)

spark

In [15]:
# 04 ... ETL ... read CSV file into Pandas dataframe
df_rejected = pandas.read_csv("rejected_2007_to_2018Q4.csv")
df_rejected.dtypes

Amount Requested        float64
Application Date         object
Loan Title               object
Risk_Score              float64
Debt-To-Income Ratio     object
Zip Code                 object
State                    object
Employment Length        object
Policy Code             float64
dtype: object

In [16]:
# 05 ... ETL transform DF into proper DF for ML
df_rejected.columns = ['loan_amount', 'loan_date', 'loan_description', 'loan_risk', 'applicant_debt_income_ratio', 'applicant_zipcode', 'applicant_state','applicant_employment_length', 'loan_policy_code']
df_rejected.replace({'applicant_employment_length':'years'},'',regex=True,inplace=True)
df_rejected.replace({'applicant_employment_length':'year'},'',regex=True,inplace=True)
df_rejected.replace({'applicant_employment_length':'< 1'},'0',regex=True,inplace=True)
df_rejected.replace({'applicant_employment_length':'0+'},'1',regex=True,inplace=True)
df_rejected.replace({'applicant_employment_length':'10+'},'10',regex=True,inplace=True)
df_rejected.replace({'applicant_employment_length':'0+'},'1',regex=True,inplace=True)
df_rejected.replace({'applicant_debt_income_ratio':'%'},'',regex=True,inplace=True)

df_rejected

,loan_amount,loan_date,loan_description,loan_risk,applicant_debt_income_ratio,applicant_zipcode,applicant_state,applicant_employment_length,loan_policy_code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,1,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,1,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,1,0.0
...,...,...,...,...,...,...,...,...,...
27648736,10000.0,2016-12-31,Debt consolidation,590.0,41.26%,441xx,OH,1,0.0
27648737,10000.0,2016-12-31,moving,NaN,1.48%,207xx,MD,5,0.0
27648738,1200.0,2016-12-31,Other,686.0,10.26%,914xx,CA,1,0.0
27648739,25000.0,2016-12-31,debt_consolidation,NaN,17.71%,880xx,NM,1,0.0


In [29]:
# 06 .. ETL transform DF into proper DF for ML
df_rejected.astype({'applicant_debt_income_ratio':'float64'}).dtypes
df_rejected.dtypes
df_rejected.to_csv('lending_club_loans_rejected.csv', sep=',')
df_rejected

,loan_amount,loan_date,loan_description,loan_risk,applicant_debt_income_ratio,applicant_zipcode,applicant_state,applicant_employment_length,loan_policy_code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10,481xx,NM,4,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10,010xx,MA,1,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10,212xx,MD,1,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64,017xx,MA,1,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43,209xx,MD,1,0.0
...,...,...,...,...,...,...,...,...,...
27648736,10000.0,2016-12-31,Debt consolidation,590.0,41.26,441xx,OH,1,0.0
27648737,10000.0,2016-12-31,moving,NaN,1.48,207xx,MD,5,0.0
27648738,1200.0,2016-12-31,Other,686.0,10.26,914xx,CA,1,0.0
27648739,25000.0,2016-12-31,debt_consolidation,NaN,17.71,880xx,NM,1,0.0


In [30]:
#07 ... PySpark SQLContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

sparkSqlContext = SQLContext(spark)
df_rejected_sql = sparkSqlContext.read.csv('lending_club_loans_rejected.csv', header=True,inferSchema=True,sep=',',mode='PERMISSIVE')
df_rejected_sql

DataFrame[_c0: int, loan_amount: double, loan_date: timestamp, loan_description: string, loan_risk: string, applicant_debt_income_ratio: string, applicant_zipcode: string, applicant_state: string, applicant_employment_length: string, loan_policy_code: string]

In [31]:
#08a .. PySpark ... PySparkDataFrame
print('Rows: ' + str(df_rejected_sql.count()))
df_rejected_sql.dtypes

Rows: 27648741


[('_c0', 'int'),
 ('loan_amount', 'double'),
 ('loan_date', 'timestamp'),
 ('loan_description', 'string'),
 ('loan_risk', 'string'),
 ('applicant_debt_income_ratio', 'string'),
 ('applicant_zipcode', 'string'),
 ('applicant_state', 'string'),
 ('applicant_employment_length', 'string'),
 ('loan_policy_code', 'string')]

In [36]:
#08b .. PySpark .. PySparkDataFrame
df_rejected_sql.show(10)

+---+-----------+-------------------+--------------------+---------+---------------------------+-----------------+---------------+---------------------------+----------------+
|_c0|loan_amount|          loan_date|    loan_description|loan_risk|applicant_debt_income_ratio|applicant_zipcode|applicant_state|applicant_employment_length|loan_policy_code|
+---+-----------+-------------------+--------------------+---------+---------------------------+-----------------+---------------+---------------------------+----------------+
|  0|     1000.0|2007-05-26 00:00:00|Wedding Covered b...|    693.0|                         10|            481xx|             NM|                         4 |             0.0|
|  1|     1000.0|2007-05-26 00:00:00|  Consolidating Debt|    703.0|                         10|            010xx|             MA|                         1 |             0.0|
|  2|    11000.0|2007-05-27 00:00:00|Want to consolida...|    715.0|                         10|            212xx|      

In [33]:
# 08c ... PySpark SQLContext
df_rejected_sql.registerTempTable("rejected_loans")

In [45]:
# 08d
df_rejected_texas_sql = sparkSqlContext.sql("SELECT * FROM rejected_loans WHERE loan_amount IS NOT NULL AND loan_risk IS NOT NULL AND applicant_state='TX'")
df_rejected_texas_sql.show(10)

+---+-----------+-------------------+----------------+---------+---------------------------+-----------------+---------------+---------------------------+----------------+
|_c0|loan_amount|          loan_date|loan_description|loan_risk|applicant_debt_income_ratio|applicant_zipcode|applicant_state|applicant_employment_length|loan_policy_code|
+---+-----------+-------------------+----------------+---------+---------------------------+-----------------+---------------+---------------------------+----------------+
| 16|    12000.0|2007-05-29 00:00:00|FoundersCafe.com|    685.0|                         10|            770xx|             TX|                         3 |             0.0|
| 20|    15000.0|2007-05-29 00:00:00|           twbmc|    712.0|                         10|            773xx|             TX|                         1 |             0.0|
| 35|     6500.0|2007-05-30 00:00:00|         natbunn|    632.0|                      12.98|            773xx|             TX|              

In [35]:
#08e
df_rejected_sql.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- loan_amount: double (nullable = true)
 |-- loan_date: timestamp (nullable = true)
 |-- loan_description: string (nullable = true)
 |-- loan_risk: string (nullable = true)
 |-- applicant_debt_income_ratio: string (nullable = true)
 |-- applicant_zipcode: string (nullable = true)
 |-- applicant_state: string (nullable = true)
 |-- applicant_employment_length: string (nullable = true)
 |-- loan_policy_code: string (nullable = true)



In [46]:
#08f
# https://www.datacamp.com/cheat-sheet/pyspark-cheat-sheet-spark-dataframes-in-python
# http://datacamp-community-prod.s3.amazonaws.com/02213cb4-b391-4516-adcd-57243ced8eed
# https://www.analyticsvidhya.com/blog/2021/05/9-most-useful-functions-for-pyspark-dataframe/
df_rejected_texas_sql.groupBy("loan_risk", "loan_amount").count().orderBy("count",ascending=0).show()

+---------+-----------+-----+
|loan_risk|loan_amount|count|
+---------+-----------+-----+
|    501.0|     1000.0| 4298|
|    501.0|     5000.0| 2046|
|    501.0|     2000.0| 1406|
|      0.0|     1000.0| 1331|
|    643.0|    10000.0| 1280|
|    620.0|    10000.0| 1209|
|    501.0|    10000.0| 1128|
|    573.0|     1000.0| 1084|
|    501.0|     3000.0| 1005|
|    524.0|     1000.0|  969|
|    620.0|    20000.0|  944|
|    643.0|     5000.0|  928|
|    652.0|    10000.0|  912|
|    501.0|     1500.0|  898|
|    573.0|     5000.0|  893|
|    620.0|     5000.0|  891|
|    650.0|    10000.0|  860|
|    653.0|    10000.0|  855|
|    651.0|    10000.0|  852|
|    647.0|    10000.0|  815|
+---------+-----------+-----+
only showing top 20 rows

